# Twitter Data Scraping with Tweepy

This notebook demonstrates how to scrape recent tweets using the Tweepy API, process them, and save the results into a CSV file. The steps include:
1. **Setup and Authentication**: Configuring Tweepy with secure API keys.
2. **Defining a Function to Retrieve Tweets**: Using the Tweepy client to fetch tweets containing a specific hashtag.
3. **Storing and Saving the Data**: Converting the fetched tweets into a DataFrame and saving them to a CSV file.

> **Note**: API keys and tokens are handled securely using environment variables.

In [1]:
# Import the packages and modules
from dotenv import load_dotenv
import pandas as pd
import tweepy as tp
import os 

/Users/hashimkhan/Documents/greenbootcamps/sentiment_analysis_X/sentimentAnalysis/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:

# Load API keys from .env file
load_dotenv()

api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")
access_token = os.getenv("ACCESS_TOKEN")
access_token_secret = os.getenv("ACCESS_TOKEN_SECRET")

# Initialize Tweepy client
client = tp.Client(bearer_token=bearer_token)

### Secure Authentication

API keys and tokens are loaded securely from a `.env` file. This approach ensures sensitive data is not exposed in the code.


### Function to Retrieve Tweets by Hashtag
The following function uses Tweepy to fetch recent tweets containing a specific hashtag.
- **Query Format**: Only tweets with the given hashtag are fetched, excluding retweets.
- **Fields Included**: Tweet text, author information, location, and public metrics like retweets and likes.


In [ ]:
def get_tweets_by_hashtag(hashtag, max_results=60):
    """
    Retrieve tweets containing a specific hashtag.

    Parameters:
    - hashtag (str): The hashtag to search for (without the # symbol).
    - max_results (int): Maximum number of tweets to retrieve.

    Returns:
    - list: A list of dictionaries containing tweet data.
    """
    tweets_data = []
    query = f"#{hashtag} -is:retweet"  # Exclude retweets
    
    try:
        tweets = client.search_recent_tweets(
            query=query,
            max_results=min(max_results, 100),
            tweet_fields=["created_at", "text", "author_id", "lang", "public_metrics"],
            user_fields=["username", "name", "location"],
            expansions=["author_id"],
        )
        
        if tweets.data:
            users = {user["id"]: user for user in tweets.includes["users"]}  # Map user data
            for tweet in tweets.data:
                user = users.get(tweet.author_id, {})
                tweets_data.append({
                    "username": user.get("username"),
                    "name": user.get("name"),
                    "location": user.get("location"),
                    "tweet_text": tweet.text,
                    "created_at": tweet.created_at,
                    "retweets": tweet.public_metrics.get("retweet_count"),
                    "likes": tweet.public_metrics.get("like_count"),
                    "language": tweet.lang,
                })
    except tp.TweepyException as e:
        print(f"Error: {e}")
    
    return tweets_data

### Usage (Call the above function according to the hashtag of your choice)

In [4]:
# Example usage
hashtag = "The_hashtag_without_#_sign"
tweets = get_tweets_by_hashtag(hashtag, max_results=100)

Error: 429 Too Many Requests
Usage cap exceeded: Monthly product cap


### Save Tweets to CSV

The fetched tweet data is stored in a Pandas DataFrame and saved as a CSV file for further analysis.


In [ ]:
# Convert the tweet data to a DataFrame
df = pd.DataFrame(tweets)

# Save the DataFrame to a CSV file
output_file = "file_name.csv"
df.to_csv(output_file, index=False, encoding="utf-8")

# Display a summary of the data
print(f"Data saved to {output_file}")
print(f"Number of tweets fetched: {len(df)}")
print(df.head(10))